In [ ]:
!pip install --upgrade transformers
!pip install -q datasets

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Load your CSV
df = pd.read_csv("/content/TROCR-FINAL-RESULT.csv")
df = df[["Prescription", "Appointment"]].dropna().reset_index(drop=True)
df.columns = ["input_text", "target_text"]

# Split into train and test (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
# Step 3: Convert to Hugging Face Dataset
from datasets import Dataset

dataset = Dataset.from_pandas(df)

# Step 4: Tokenization
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Define the max token lengths
max_input_length = 128
max_target_length = 128

# Tokenization function
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=max_input_length,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["target_text"],
            max_length=max_target_length,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/988 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/247 [00:00<?, ? examples/s]

In [ ]:
from transformers import BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./mbart-correction",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    logging_dir="./logs",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: shroudmohamed1 (shroudmohamed1-ain-shams-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.010700,0.005734
2,0.051800,0.001980
3,0.001500,0.002438
4,0.000500,0.001807
5,0.002200,0.001207


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1235, training_loss=0.2277137337452336, metrics={'train_runtime': 423.0525, 'train_samples_per_second': 11.677, 'train_steps_per_second': 2.919, 'total_flos': 376512262963200.0, 'train_loss': 0.2277137337452336, 'epoch': 5.0})

In [ ]:
model.save_pretrained("/content/GroundTruthAppointbart-ocr-correction")
tokenizer.save_pretrained("/content/GroundTruthAppointbart-ocr-correction")

('/content/GroundTruthAppointbart-ocr-correction/tokenizer_config.json',
 '/content/GroundTruthAppointbart-ocr-correction/special_tokens_map.json',
 '/content/GroundTruthAppointbart-ocr-correction/vocab.json',
 '/content/GroundTruthAppointbart-ocr-correction/merges.txt',
 '/content/GroundTruthAppointbart-ocr-correction/added_tokens.json')

In [ ]:
import shutil
shutil.make_archive("/content/GroundTruthAppointbart-ocr-correction", 'zip', "/content/GroundTruthAppointbart-ocr-correction")


'/content/GroundTruthAppointbart-ocr-correction.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
shutil.move("/content/GroundTruthAppointbart-ocr-correction.zip", "/content/drive/MyDrive/GroundTruthAppointbart-ocr-correction.zip")


'/content/drive/MyDrive/GroundTruthAppointbart-ocr-correction.zip'